In [1]:
import pandas as pd
from multiprocessing import Pool, cpu_count
import numpy as np
from src.input.write_files import write_int_matrix_file
from src.input.read_files import find_files, numbers_from_file_name, normal_sec_size_int_from_file_name, number_from_text
from datetime import datetime
from src.algorithm.vanilla.circuit_synthesis import synthesise_circuit
from src.algorithm.vanilla.execution import load_matrix_and_benchmark
import csv

print(f'There are {cpu_count()} threads available on this machine.')

There are 12 threads available on this machine.


In [2]:
sample_size = 1
lower_size = 120
upper_size = 200
step_size = 10

sizes = np.arange(lower_size, upper_size + step_size, step_size)
samples = np.arange(1, sample_size + 1, 1)

input_gen_processors = 10

input_dir = r'resources\measurement\input_matrices'
input_file_prefix = 'SingMat'

benchmark_processors = 10

output_dir = r'resources\measurement'
output_file_prefix = 'CircuitSizeMeasurement'

output_file_fieldnames = ['size', 'run', 'sec_size', 'num_gates', 'process_time', 'initial_rss', 'final_rss']

In [3]:
input_gen_argument_tuples = ([(size, size * size, rf'..\{input_dir}\{input_file_prefix}_{size}_{sample}.txt') for size in sizes for sample in samples])

[print(argument_tuple) for argument_tuple in input_gen_argument_tuples[:min(10, len(input_gen_argument_tuples))]]

(120, 14400, '..\\resources\\measurement\\input_matrices\\SingMat_120_1.txt')
(130, 16900, '..\\resources\\measurement\\input_matrices\\SingMat_130_1.txt')
(140, 19600, '..\\resources\\measurement\\input_matrices\\SingMat_140_1.txt')
(150, 22500, '..\\resources\\measurement\\input_matrices\\SingMat_150_1.txt')
(160, 25600, '..\\resources\\measurement\\input_matrices\\SingMat_160_1.txt')
(170, 28900, '..\\resources\\measurement\\input_matrices\\SingMat_170_1.txt')
(180, 32400, '..\\resources\\measurement\\input_matrices\\SingMat_180_1.txt')
(190, 36100, '..\\resources\\measurement\\input_matrices\\SingMat_190_1.txt')
(200, 40000, '..\\resources\\measurement\\input_matrices\\SingMat_200_1.txt')


[None, None, None, None, None, None, None, None, None]

In [4]:
print(f'Generating {len(input_gen_argument_tuples)} matrices in size range from {lower_size} to {upper_size} ...')

if __name__ ==  '__main__':
    with Pool(processes = input_gen_processors) as pool:
        pool.starmap(write_int_matrix_file, input_gen_argument_tuples)

print(f'Saved matrices to {input_dir} ...')

Generating 9 matrices in size range from 120 to 200 ...
Saved matrices to resources\measurement\input_matrices ...


In [5]:
file_regex = rf'({input_file_prefix}).*\.txt'
size_regex = r'_\d+_'
sample_regex = r'_\d+\.'

input_file_names = find_files(rf'..\{input_dir}', file_regex)
input_file_names.sort(key = lambda input_file_name: numbers_from_file_name(input_file_name, size_regex, sample_regex))

[print(input_file_name) for input_file_name in input_file_names[:min(10, len(input_file_names))]]

SingMat_120_1.txt
SingMat_130_1.txt
SingMat_140_1.txt
SingMat_150_1.txt
SingMat_160_1.txt
SingMat_170_1.txt
SingMat_180_1.txt
SingMat_190_1.txt
SingMat_200_1.txt


[None, None, None, None, None, None, None, None, None]

In [6]:
file_name_arguments = [(number_from_text(input_file_name, size_regex),
                        number_from_text(input_file_name, sample_regex),
                        input_file_name)
                       for input_file_name in input_file_names]
file_name_arguments_df = pd.DataFrame(file_name_arguments, columns=['size', 'sample', 'path'])
file_name_arguments_df.head()

,size,sample,path
0,120,1,SingMat_120_1.txt
1,130,1,SingMat_130_1.txt
2,140,1,SingMat_140_1.txt
3,150,1,SingMat_150_1.txt
4,160,1,SingMat_160_1.txt


In [7]:
section_sizes_hard = [5]

benchmark_tuples_list = [(size, sec_size, sample, path) for size, sample, path in file_name_arguments_df.values.tolist() for sec_size in section_sizes_hard]

[print(benchmark_tuple) for benchmark_tuple in benchmark_tuples_list[:min(10, len(benchmark_tuples_list))]]

(120, 5, 1, 'SingMat_120_1.txt')
(130, 5, 1, 'SingMat_130_1.txt')
(140, 5, 1, 'SingMat_140_1.txt')
(150, 5, 1, 'SingMat_150_1.txt')
(160, 5, 1, 'SingMat_160_1.txt')
(170, 5, 1, 'SingMat_170_1.txt')
(180, 5, 1, 'SingMat_180_1.txt')
(190, 5, 1, 'SingMat_190_1.txt')
(200, 5, 1, 'SingMat_200_1.txt')


[None, None, None, None, None, None, None, None, None]

In [8]:
benchmark_argument_tuples = [(synthesise_circuit,
                              size,
                              sec_size,
                              sample,
                              rf'..\{input_dir}\{input_file_name}')
                             for (size, sec_size, sample, input_file_name) in benchmark_tuples_list]

In [9]:
output_file_timestamp = datetime.now().strftime('%d%m%y_%H%M')

print(f'Measuring circuit sizes in range from {lower_size} to {upper_size} in {benchmark_processors} threads ...')

if __name__ ==  '__main__':
    with Pool(processes = benchmark_processors) as pool:
        result_tuples = pool.starmap(load_matrix_and_benchmark, benchmark_argument_tuples)

print(f'Saving results to {output_dir} ...')

with open(rf'..\{output_dir}\{output_file_prefix}_{output_file_timestamp}.csv', mode='w', newline='') as output_file:
    writer = csv.DictWriter(output_file, fieldnames=output_file_fieldnames)
    writer.writeheader()

    for (size, run, sec_size, num_gates, process_time, initial_rss, final_rss) in result_tuples:

        writer.writerow({'size': size, 'run': run, 'sec_size': sec_size, 'num_gates': num_gates, 'process_time': process_time, 'initial_rss': initial_rss, 'final_rss': final_rss})

print('Done.')




Measuring circuit sizes in range from 120 to 200 in 10 threads ...
Saving results to resources\measurement ...
Done.
